### 데이터 불균형 : 클래스가 어느 한 쪽으로만 일방적으로 존재

* 해결 방법
1) 오버 샘플링 : 
>클래스가 적은 쪽의 데이터를 랜덤샘플링하여 복사 붙여넣기를 반복하여 
                    두 클래스의 비율을 비슷하게 함.
                          
2) 언더 샘플링 : 
>클래스가 많은 쪽의 데이터를 랜덤 샘플링하여 삭제하기를 반복하여 
                    두 클래스의 비율을 비슷하게 함.

3) 오버 & 언더 샘플링
4) SMOTE 알고리즘
 >기존 데이터를 적절하게 혼합하여 새로운 데이터를 생성하는 방법

> ex) Y : 1000건 vs N : 10건 -> 1010 / 2 = 505, 
>> Y는 505건이 될때까지 언더 샘플링
>> N은 505건이 될때까지 오버 샘플링

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
card_df = pd.read_csv("creditcard.csv")
card_df.head(3)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0


* train 데이터와 test 데이터 나누기

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
x_features = card_df.iloc[:,:-1]  # x
y_target = card_df.iloc[:,-1]   # y 구하고자 하는 값 "class"

In [5]:
xtrain, xtest, ytrain, ytest = train_test_split(x_features,y_target,
                                                test_size = 0.3, random_state=20231023,
                                               stratify=y_target # 층화추출 : 비율 맞춰 추출
                                               )

In [6]:
ytrain.value_counts() # 비정상 거래 336건

0    199020
1       344
Name: Class, dtype: int64

In [7]:
ytest.value_counts()

0    85295
1      148
Name: Class, dtype: int64

In [8]:
def get_preprocessed_df(df = None):
    df_copy = df.copy()
    df_copy.drop("Time",axis = 1, inplace = True)
    return df_copy

In [9]:
df_copy = get_preprocessed_df(card_df)

In [10]:
df_copy

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,4.356170,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,-0.975926,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,-0.484782,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,-0.399126,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [11]:
def get_train_test_dataset(df=None):
    df_copy = get_preprocessed_df(df)
    X_features = df_copy.iloc[:, :-1]
    y_target = df_copy.iloc[:, -1]
    X_train, X_test, y_train, y_test = \
    train_test_split(X_features, y_target, test_size=0.3, random_state=0, stratify=y_target)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = get_train_test_dataset(card_df)

In [12]:
print(y_train.value_counts()/y_train.shape[0]*100)

0    99.827451
1     0.172549
Name: Class, dtype: float64


In [13]:
print(y_test.value_counts()/y_test.shape[0]*100)

0    99.826785
1     0.173215
Name: Class, dtype: float64


In [14]:
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression()

In [15]:
import warnings
warnings.filterwarnings("ignore")
lr_clf.fit(X_train,y_train)

LogisticRegression()

In [16]:
lr_clf.predict(X_test)
pd.Series(lr_clf.predict(X_test)).value_counts()

0    85341
1      102
dtype: int64

In [17]:
lr_clf.predict_proba(X_test)
lr_clf.predict_proba(X_test)[:,1]

array([0.00134831, 0.00012345, 0.00019178, ..., 0.00024669, 0.00074666,
       0.00013475])

# LightGbm 설치

In [18]:
pip install lightgbm

  Obtaining dependency information for lightgbm from https://files.pythonhosted.org/packages/b3/f8/ee33e36194eb03a76eccf3adac3fba51f0e56fbd20609bb531659d48d3cb/lightgbm-4.1.0-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------- ----------------- 0.7/1.3 MB 23.1 MB/s eta 0:00:01
   ---------------------------------------  1.3/1.3 MB 27.7 MB/s eta 0:00:01
   ---------------------------------------- 1.3/1.3 MB 16.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [19]:
from lightgbm import LGBMClassifier

In [20]:
lgbm_clf = LGBMClassifier(n_estimators = 1000 , num_leaves = 64, boost_from_average = False)
# boost_from_average 데이터가 불균형 분포되어 있을때 False

In [22]:
lgbm_clf.fit(X_train,y_train)

[LightGBM] [Info] Number of positive: 344, number of negative: 199020
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014465 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 199364, number of used features: 29
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits w

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [23]:
pred = lgbm_clf.predict(X_test)
pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [24]:
lgbm_clf.predict_proba(X_test)

array([[9.99999997e-01, 3.49830240e-09],
       [1.00000000e+00, 1.08517297e-10],
       [1.00000000e+00, 2.23232317e-10],
       ...,
       [9.99999996e-01, 3.66393546e-09],
       [9.99999999e-01, 6.63996306e-10],
       [1.00000000e+00, 1.08138237e-10]])

In [25]:
X_train.describe()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
count,199364.000000,199364.000000,199364.000000,199364.000000,199364.000000,199364.000000,199364.000000,199364.000000,199364.000000,199364.000000,...,199364.000000,199364.000000,199364.000000,199364.000000,199364.000000,199364.000000,199364.000000,199364.000000,199364.000000,199364.000000
mean,0.000386,-0.000637,0.000587,-0.001239,0.000596,0.000806,-0.000127,0.001900,0.000387,-0.000203,...,0.000546,0.001181,0.000770,-0.000907,0.000708,0.000160,-0.001173,-0.000476,0.000252,88.286313
std,1.959976,1.658734,1.512912,1.413217,1.362707,1.322962,1.216585,1.191216,1.098032,1.086637,...,0.774205,0.736703,0.724805,0.630090,0.605543,0.520830,0.482453,0.401370,0.330830,248.033917
min,-56.407510,-72.715728,-32.965346,-5.683171,-35.182120,-26.160506,-43.557242,-73.216718,-13.434066,-24.588262,...,-28.009635,-34.830382,-10.933144,-44.807735,-2.836627,-10.295397,-2.604551,-22.565679,-11.710896,0.000000
25%,-0.921490,-0.598848,-0.889918,-0.849806,-0.690631,-0.767513,-0.556268,-0.207917,-0.642713,-0.535148,...,-0.211459,-0.227817,-0.540206,-0.162056,-0.353481,-0.317180,-0.328089,-0.070961,-0.053200,5.640000
50%,0.015893,0.065727,0.180635,-0.019842,-0.054758,-0.273142,0.040109,0.023008,-0.050546,-0.091521,...,-0.062514,-0.028990,0.007585,-0.011301,0.041293,0.016244,-0.053516,0.001207,0.011043,22.000000
75%,1.316633,0.804333,1.028314,0.744065,0.609794,0.400874,0.570132,0.327720,0.597585,0.456096,...,0.133886,0.186863,0.529286,0.147035,0.440205,0.351141,0.240123,0.090888,0.078216,77.000000
max,2.454930,22.057729,9.382558,16.875344,34.801666,21.550496,36.877368,20.007208,15.594995,23.745136,...,39.420904,27.202839,10.503090,22.528412,4.022866,6.070850,3.463246,12.152401,33.847808,19656.530000


In [31]:
# 이상치 제거
# 표준화
q25 = np.percentile(X_train["V1"].values, 25)
q75 = np.percentile(X_train["V1"].values, 75)
iqr = q75 - q25
iqr15 = iqr*1.5
lower_fence = q25-iqr15 # Lower Fence
upper_fence = q75+iqr15 # Upper Fence

In [33]:
X_train["V1"][(X_train["V1"] < lower_fence) | (X_train["V1"] > upper_fence)] # 아웃라이어

211605    -8.367621
151631    -7.824182
76190     -6.078681
207772    -7.794335
220170    -5.121007
            ...    
125442    -8.836906
264145    -4.692089
216442   -16.536406
20260    -10.648382
19760    -14.191832
Name: V1, Length: 4928, dtype: float64

# Smote 오버샘플링

In [35]:
pip install -U imbalanced-learn

Note: you may need to restart the kernel to use updated packages.
